In [ ]:
CREATE TABLE IF NOT EXISTS DATA_SLV.DIM_GROUPS (
    ID_GROUP NUMBER(38,0),
    ID_CATEGORY NUMBER(38,0),
    ID_CITY NUMBER(38,0),
    ID_MEMBER NUMBER(38,0),
    CREATED TIMESTAMP_NTZ,
    DESCRIPTION VARCHAR,
    ID_PHOTO NUMBER(38,0),
    JOIN_MODE VARCHAR,
    LINK VARCHAR,
    MEMBERS NUMBER(38,0),
    GROUP_NAME VARCHAR,
    RATING NUMBER(38,0),
    STATE VARCHAR,
    TIMEZONE VARCHAR,
    URLNAME VARCHAR,
    UTC_OFFSET NUMBER(38,0),
    VISIBILITY VARCHAR,
    WHO VARCHAR
);

MERGE INTO DATA_SLV.DIM_GROUPS AS d
USING (
  SELECT DISTINCT
    TRY_TO_NUMBER(g.GROUP_ID)                     AS ID_GROUP,
    TRY_TO_NUMBER(g.CATEGORY_ID)                  AS ID_CATEGORY,
    TRY_TO_NUMBER(g.CITY_ID)                      AS ID_CITY,
    g.CREATED                                     AS CREATED,
    NULLIF(TRIM(g.DESCRIPTION), '')               AS DESCRIPTION,
    TRY_TO_NUMBER(g."group_photo.photo_id")       AS ID_PHOTO,
    NULLIF(TRIM(g.JOIN_MODE),   '')               AS JOIN_MODE,
    NULLIF(TRIM(g.LINK),        '')               AS LINK,
    TRY_TO_NUMBER(g.MEMBERS)                      AS MEMBERS,
    NULLIF(TRIM(g.GROUP_NAME),  '')               AS GROUP_NAME,
    TRY_TO_NUMBER(ROUND(g.RATING, 0))             AS RATING,          -- tu destino es NUMBER(38,0)
    NULLIF(TRIM(g.STATE),       '')               AS STATE,
    NULLIF(TRIM(g.TIMEZONE),    '')               AS TIMEZONE,
    NULLIF(TRIM(g.URLNAME),     '')               AS URLNAME,
    TRY_TO_NUMBER(g.UTC_OFFSET)                   AS UTC_OFFSET,
    NULLIF(TRIM(g.VISIBILITY),  '')               AS VISIBILITY,
    NULLIF(TRIM(g.WHO),         '')               AS WHO,
    TRY_TO_NUMBER(g."organizer.member_id")        AS ID_MEMBER
  FROM MEETUP_DB.DATA_BRZ.GROUPS g
  WHERE TRY_TO_NUMBER(g.GROUP_ID) IS NOT NULL
) AS s
ON d.ID_GROUP = s.ID_GROUP

WHEN MATCHED THEN UPDATE SET
  d.ID_CATEGORY  = s.ID_CATEGORY,
  d.ID_CITY      = s.ID_CITY,
  d.ID_MEMBER    = s.ID_MEMBER,
  d.CREATED      = s.CREATED,
  d.DESCRIPTION  = s.DESCRIPTION,
  d.ID_PHOTO     = s.ID_PHOTO,
  d.JOIN_MODE    = s.JOIN_MODE,
  d.LINK         = s.LINK,
  d.MEMBERS      = s.MEMBERS,
  d.GROUP_NAME   = s.GROUP_NAME,
  d.RATING       = s.RATING,
  d.STATE        = s.STATE,
  d.TIMEZONE     = s.TIMEZONE,
  d.URLNAME      = s.URLNAME,
  d.UTC_OFFSET   = s.UTC_OFFSET,
  d.VISIBILITY   = s.VISIBILITY,
  d.WHO          = s.WHO

WHEN NOT MATCHED THEN INSERT (
  ID_GROUP, ID_CATEGORY, ID_CITY, ID_MEMBER, CREATED, DESCRIPTION, ID_PHOTO,
  JOIN_MODE, LINK, MEMBERS, GROUP_NAME, RATING, STATE, TIMEZONE, URLNAME,
  UTC_OFFSET, VISIBILITY, WHO
) VALUES (
  s.ID_GROUP, s.ID_CATEGORY, s.ID_CITY, s.ID_MEMBER, s.CREATED, s.DESCRIPTION, s.ID_PHOTO,
  s.JOIN_MODE, s.LINK, s.MEMBERS, s.GROUP_NAME, s.RATING, s.STATE, s.TIMEZONE, s.URLNAME,
  s.UTC_OFFSET, s.VISIBILITY, s.WHO
);